In [1]:
import sys

sys.path.append('../GraphStructureLearning')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import pickle

from torch_geometric.data import Data as gData
from torch_geometric.utils import to_networkx, to_undirected
from torch_geometric.nn import MessagePassing

import networkx as nx
import matplotlib.pyplot as plt

from torch_geometric_temporal.nn.recurrent import DCRNN

# GTS

In [3]:
from models.GTS.gts_graph_learning import GTS_Graph_Learning
from models.GTS.gts_forecasting_module import GTS_Forecasting_Module
from models.GTS.DCRNN import DCRNN
from utils.utils import build_fully_connected_edge_idx

from glob import glob
import yaml
from easydict import EasyDict as edict

C:\Users\KimJuHyeon\GSL\utils\utils.py:22: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(edge_index) is not 1:


In [4]:
config_file = glob('./config/GTS/*.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [5]:
# node_feas = torch.rand(config.nodes_num, config.node_features, 1000)
# edge_index = build_fully_connected_edge_idx(num_nodes=config.nodes_num)

In [6]:
spike = pickle.load(open('./data/LNP_spk_all.pickle', 'rb'))

spike = torch.FloatTensor(spike[:,:1000])

In [7]:
config.nodes_num = 100

In [14]:
edge_index = build_fully_connected_edge_idx(num_nodes=config.nodes_num)

In [21]:
edge_index.shape

torch.Size([2, 9900])

In [22]:
spike.shape

torch.Size([100, 1000])

In [12]:
gl = GTS_Graph_Learning(config)

In [15]:
adj = gl(spike, edge_index)

In [18]:
z_1 = F.gumbel_softmax(adj, tau=0.3, hard=True)
z_1 = torch.transpose(z_1, 0, 1)

In [19]:
edge_ = []

for ii, rel in enumerate(z_1[0]):
    if bool(rel):
        edge_.append(edge_index[:, ii])

adj_matrix = torch.stack(edge_, dim=-1)

In [20]:
adj_matrix.shape

torch.Size([2, 3867])

In [23]:
config.num_layer=2

In [24]:
config.embedding_dim = 13
config.in_channels = 1

In [25]:
encoder_dcrnn = DCRNN(config)
decoder_dcrnn = DCRNN(config)

In [27]:
inputs = spike.reshape(-1,100,1,100)
inputs.shape

torch.Size([10, 100, 1, 100])

In [28]:
encoder_hidden_state = encoder_dcrnn(inputs[0,:,:,0], adj_matrix)

In [29]:
encoder_hidden_state_2 = encoder_dcrnn(inputs[0,:,:,1], adj_matrix, hidden_state=encoder_hidden_state)

In [30]:
decoder_hidden_state = decoder_dcrnn(inputs[0,:,:,3], adj_matrix, hidden_state=encoder_hidden_state_2)

In [31]:
decoder_hidden_state[-1].shape

torch.Size([100, 64])

In [32]:
forecasting = GTS_Forecasting_Module(config)

In [33]:
encoder_inputs = torch.rand(config.nodes_num, config.node_features, 70)
decoder_inputs = torch.rand(config.nodes_num, config.node_features, 30)

In [15]:
a = torch.rand(2, 1)
b = torch.rand(2, 1)

In [16]:
output = []

output.append(a)
output.append(b)

In [17]:
torch.stack(output, dim=-1).shape

torch.Size([2, 1, 2])

In [18]:
torch.concat([a,b], dim=-1).shape

torch.Size([2, 2])

In [19]:
a

tensor([[0.2259],
        [0.2618]])

In [20]:
b

tensor([[0.6195],
        [0.2255]])

In [21]:
torch.concat([a,b], dim=-1)

tensor([[0.2259, 0.6195],
        [0.2618, 0.2255]])

In [22]:
outputs = []

for i in range(20):
    output = torch.rand(2, 1)
    outputs.append(output)

In [25]:
torch.cat(outputs,dim=-1).shape

torch.Size([2, 20])

In [26]:
np.random.random()

0.13928512732914022

In [60]:
np.random.random()

0.45935605045067873

In [6]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

In [8]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        print(x.shape)
        h = self.recurrent(x, edge_index, edge_weight)
        print(h.shape)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [14]:
train_dataset

In [54]:
snapshot.x

tensor([[-5.9412e-01, -1.0814e-03,  9.4778e-01, -2.9760e-01],
        [-1.1995e+00,  6.7890e-01,  9.0432e-01, -9.3655e-01],
        [ 5.3065e-01,  1.2104e+00, -1.6286e+00,  2.3300e+00],
        [-3.2629e-01,  2.1240e-01,  7.5109e-01,  2.0564e+00],
        [-4.0526e-01,  8.6434e-01,  4.7727e-01, -8.0782e-01],
        [ 6.2654e-01, -6.4068e-02,  2.3037e+00, -2.5305e+00],
        [ 1.3063e-01,  2.1869e+00, -1.2402e+00, -1.6031e+00],
        [-2.3423e-01,  1.1811e+00, -2.3423e-01, -1.1629e-01],
        [-1.0827e-01,  1.1149e+00, -1.6516e-01, -2.7894e-01],
        [-9.5250e-01,  2.3186e+00,  2.9811e+00, -4.4306e+00],
        [-1.4333e+00,  1.6566e+00,  2.5876e-01, -1.3230e+00],
        [-3.4669e-01,  4.4487e-01,  1.4343e+00, -1.5835e+00],
        [-2.3837e+00,  1.5276e+00,  4.0118e+00, -2.6479e+00],
        [-1.1852e+00,  2.2989e+00,  5.3610e-01, -1.8074e+00],
        [-2.6931e+00,  2.4792e+00,  8.8681e-02, -2.0411e+00],
        [-1.2430e-01,  1.5093e-03,  1.6371e+00, -4.7029e-01],
        

In [57]:
snapshot.edge_index

tensor([[ 0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,
          3,  3,  3,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  7,  7,
          7,  7,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10,
         10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13,
         13, 14, 14, 14, 14, 14, 14, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17,
         17, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19],
        [10,  6, 13,  1,  0,  5, 16,  0, 16,  1, 14, 10,  8,  2,  5,  8, 15, 12,
          9, 10,  3,  4, 13,  0, 10,  2,  5,  0, 16,  6, 14, 13, 11, 18,  7, 17,
         11, 18,  3,  2, 15,  8, 10,  9, 13,  3, 12, 10,  5,  9,  8,  3, 10,  2,
         13,  0,  6, 11,  7, 13, 18,  3,  9, 13, 12, 13,  9,  6,  4, 12,  0, 11,
         10, 18, 19,  1, 14,  6, 16,  3, 15,  8, 16, 14,  1,  0,  6,  7, 19, 17,
         18, 14, 18, 17,  7,  6, 19, 11, 18, 14, 19, 17]])

In [9]:
from tqdm import tqdm

model = RecurrentGCN(node_features = 4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(1)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 

100%|██████████| 1/1 [00:00<00:00,  2.92it/s]

torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 4])
torch.Size([20, 32])
torch.Size([20, 

In [11]:
y_hat.shape

torch.Size([20, 1])

In [18]:
import torch.nn as nn

In [46]:
class EncoderModel(nn.Module):
    def __init__(self, kernel_size, stride, conv1_dim, conv2_dim, fc_dim):
        super(EncoderModel, self).__init__()
        self.nodes_feas = 1
        self.kernel_size = kernel_size
        self.stride = stride
        self.conv1_dim = conv1_dim
        self.conv2_dim = conv2_dim
        self.fc_dim = fc_dim
        
        self.conv1 = torch.nn.Conv1d(self.nodes_feas, self.conv1_dim, self.kernel_size, stride=self.stride)
        self.conv2 = torch.nn.Conv1d(self.conv1_dim, self.conv2_dim, self.kernel_size, stride=self.stride)
        self.fc = torch.nn.Linear(self.fc_dim, 1)
        
        self.bn1 = torch.nn.BatchNorm1d(self.conv1_dim)
        self.bn2 = torch.nn.BatchNorm1d(self.conv2_dim)
        

    def forward(self, inputs):
        if len(inputs.shape) == 2:
            inputs = inputs.reshape(98, 1, -1)
            
        x = self.conv1(inputs)
        x = F.relu(x)
        print(x.shape)
        x = self.bn1(x)
        x = self.conv2(x)
        x = F.relu(x)
        print(x.shape)
        x = self.bn2(x)
        
        x = x.view(98, -1)
        
        x = self.fc(x)
        x = F.relu(x)
        print(x.shape)        
        
        return x

In [47]:
input = torch.Tensor(98,212)

In [50]:
embedding = EncoderModel(20, 5, 8, 16, 64)

In [51]:
out = embedding(input)

torch.Size([98, 8, 39])
torch.Size([98, 16, 4])
torch.Size([98, 1])
